# Project

### Goal: Improve our original estimate of the log error by using clustering methodologies.

## Acquisition, Prep, and Initial Exploration

Using the notebook and files you created during the exercises make any changes, additions, etc. you want at this point. NOTE: You will NOT be splitting into train and test at this point.

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Impute missing values
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format
pd.set_option('display.max_columns', None, 'display.max_rows', None)

import acquire
import wrangle_zillow
import wrangle_zillow_r

In [2]:
# zillow = acquire.get_zillow(acquire.sql)
# Save file to csv so I can quit asking the server for this data
# zillow.to_csv('zillow.csv')

In [3]:
zillow = pd.read_csv('zillow.csv', index_col='id')
# drop extra column that comes in from csv files
# zillow = zillow.drop(columns='Unnamed: 0')

In [4]:
orig_rows = zillow.shape[0]
orig_cols = zillow.shape[1]
print (f'There are {orig_rows} rows and {orig_cols} columns in the starting dataframe.')

There are 77575 rows and 67 columns in the starting dataframe.


In [5]:
# Drop rows:
# Restrict df to only properties that meet single use criteria
single_use = [261, 262, 263, 264, 266, 268, 273, 276, 279]
zillow = zillow[zillow.propertylandusetypeid.isin(single_use)]

In [6]:
# Restrict df to only those properties with at least 1 bath & bed
zillow = zillow[(zillow.bedroomcnt > 0) & (zillow.bathroomcnt > 0)]

In [7]:
print (f'There are now {zillow.shape[0]} rows in the zillow dataframe.')

There are now 71406 rows in the zillow dataframe.


In [8]:
# Change the Y in taxdelinquencyflag to 1
zillow.taxdelinquencyflag = np.where(zillow.taxdelinquencyflag == 'Y', 1, 0)
zillow.taxdelinquencyflag.value_counts()

0    68807
1     2599
Name: taxdelinquencyflag, dtype: int64

In [9]:
# Add column for counties
zillow['county'] = np.where(zillow.fips == 6037, 'Los_Angeles',
                            np.where(zillow.fips == 6059, 'Orange', 'Ventura'))

In [10]:
# Address that zip code with too many digits
zillow[zillow.regionidzip > 100000].regionidzip.value_counts()

399,675.00    12
Name: regionidzip, dtype: int64

In [11]:
# pretty sure its just an extra digit
# check if 99675 is a zipcode: nope, so its probably a typo
zillow[zillow.regionidzip == 99675].regionidzip.value_counts()

Series([], Name: regionidzip, dtype: int64)

In [12]:
zillow['regionidzip'] = zillow['regionidzip'].replace(399675, 99675)

### Ideas:

1. Data types:

Write a function that takes in a dataframe and a list of column names and returns the dataframe with the datatypes of those columns changed to a non-numeric type.
Use this function to appropriately transform any numeric columns that should not be treated as numbers.

In [14]:
def numeric_to_object(df, num_cols):
    """
    Takes in a dataframe and a list of the columns to be transformed. 
    Changes the type of each column in the list to object type.
    """
    for col in num_cols:
        df[col] = df[col].astype('int')
        df[col] = df[col].astype('object')
    return df

In [15]:
# added this function to be sure numeric columns that should be integers and not floats are treated as such
def numeric_to_int(df, num_cols):
    """
    Takes in a dataframe and a list of the columns to be transformed. 
    Changes the type of each column in the list to integer type.
    """
    for col in num_cols:
        df[col] = df[col].astype('int')
    return df

Write a function that accepts the zillow data frame and returns the data frame with the missing values filled in.

In [16]:
def fill_nulls_with_zero(df, col_names):
    for col in col_names:
        df[col] = df[col].fillna(0)
    return df

In [17]:
# Test function
null_cols = ['airconditioningtypeid', 'basementsqft', 'decktypeid',
             'fireplacecnt', 'garagecarcnt', 'garagetotalsqft',
             'hashottuborspa', 'lotsizesquarefeet', 'poolcnt',
             'poolsizesum', 'taxdelinquencyyear']

zillow = fill_nulls_with_zero(zillow, null_cols)

3. Missing Values: Of the remaining missing values, can they be imputed or otherwise estimated?

Impute those that can be imputed with the method you feel best fits the attribute.

In [18]:
# Impute location columns and info
zillow = wrangle_zillow_r.zillow_impute_knn(zillow)

In [19]:
# Heatingorsystemtypeid - None type is 13
# Assume number of stories is 1
# Assume unitcnt is 1

def handle_other_nulls(df):
    df.heatingorsystemtypeid.fillna(13, inplace=True)
    df.numberofstories.fillna(1, inplace=True)
    df.unitcnt.fillna(1, inplace=True)
    return df

In [20]:
# Test function

zillow = handle_other_nulls(zillow)

In [21]:
# pre-emptively drop unnecessary columns
zillow = wrangle_zillow.remove_columns(zillow, 
                                       ['parcelid',
                                        'architecturalstyletypeid',
                                        'buildingclasstypeid',
                                        'finishedsquarefeet13',
                                        'finishedsquarefeet15',
                                        'finishedsquarefeet50',
                                        'finishedsquarefeet6',
                                        'finishedfloor1squarefeet',
                                        'pooltypeid10',
                                        'pooltypeid2',
                                        'pooltypeid7',
                                        'fireplaceflag',
                                        'airconditioningdesc',
                                        'storydesc',
                                        'heatingorsystemdesc',
                                        'architecturalstyledesc',
                                        'buildingclassdesc',
                                        'typeconstructiondesc',
                                        'yardbuildingsqft17',
                                        'yardbuildingsqft26',
                                        'calculatedbathnbr',
                                        'fullbathcnt',
                                        'threequarterbathnbr',
                                        'typeconstructiontypeid',
                                        'storytypeid',
                                        'propertyzoningdesc', 
                                        'calculatedfinishedsquarefeet', 
                                        'regionidneighborhood',
                                        'regionidcity',
                                        'regionidcounty',
                                        'propertylandusetypeid',
                                        'rawcensustractandblock',
                                        'propertylandusedesc',
                                        'assessmentyear',
                                        'fips'])

In [22]:
zillow.isna().sum().sort_values(ascending = False)

buildingqualitytypeid         27187
finishedsquarefeet12            177
structuretaxvaluedollarcnt      107
yearbuilt                        42
taxamount                         5
landtaxvaluedollarcnt             1
taxvaluedollarcnt                 1
fireplacecnt                      0
heatingorsystemtypeid             0
hashottuborspa                    0
garagetotalsqft                   0
garagecarcnt                      0
census_tractandblock              0
longitude                         0
decktypeid                        0
bedroomcnt                        0
bathroomcnt                       0
basementsqft                      0
latitude                          0
poolsizesum                       0
lotsizesquarefeet                 0
poolcnt                           0
regionid_zip                      0
propertycountylandusecode         0
roomcnt                           0
unitcnt                           0
numberofstories                   0
taxdelinquencyflag          

In [23]:
# Try using imputer function here
# imputer function requires only numerical columns

num_vars = list(zillow.select_dtypes('number').columns)
null_df = zillow[num_vars]

imp = IterativeImputer(random_state=423)

imp.fit(null_df)
imputed_vals = pd.DataFrame(data=imp.transform(null_df),
                            columns=num_vars)
imputed_vals.head()

,airconditioningtypeid,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,decktypeid,finishedsquarefeet12,fireplacecnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,roomcnt,unitcnt,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,logerror,haversine_distance,regionid_zip
0,0.00,0.00,2.00,3.00,8.00,0.00,"2,107.00",0.00,0.00,0.00,0.00,2.00,"34,222,559.00","-118,617,387.00","9,158.00",1.00,0.00,0.00,1.00,"1,972.00",1.00,"249,655.00","624,139.00","374,484.00","7,659.36",0.00,0.00,-0.01,87.51,"96,339.00"
1,1.00,0.00,2.00,4.00,8.00,0.00,"1,882.00",0.00,0.00,0.00,0.00,2.00,"34,220,261.00","-118,616,409.00","9,035.00",1.00,0.00,0.00,1.00,"1,972.00",1.00,"253,000.00","660,000.00","407,000.00","8,123.91",0.00,0.00,0.02,87.60,"96,339.00"
2,1.00,0.00,2.00,4.00,8.00,0.00,"1,882.00",0.00,0.00,0.00,0.00,2.00,"34,222,491.00","-118,616,854.00","9,800.00",0.00,0.00,0.00,1.00,"1,972.00",1.00,"257,591.00","542,923.00","285,332.00","6,673.24",0.00,0.00,0.08,87.47,"96,339.00"
3,0.00,0.00,2.00,3.00,8.00,0.00,"1,477.00",0.00,0.00,0.00,0.00,2.00,"34,221,864.00","-118,615,739.00","11,285.00",1.00,0.00,0.00,1.00,"1,960.00",1.00,"57,968.00","78,031.00","20,063.00","1,116.46",0.00,0.00,-0.04,87.44,"96,339.00"
4,0.00,0.00,2.00,4.00,8.00,0.00,"1,918.00",0.00,0.00,0.00,0.00,2.00,"34,220,619.00","-118,615,253.00","11,239.00",1.00,0.00,0.00,1.00,"1,960.00",1.00,"167,869.00","415,459.00","247,590.00","5,239.85",0.00,0.00,-0.01,87.49,"96,339.00"


In [24]:
# Build imputer function

def zillow_imputer(df):
    num_vars = list(df.select_dtypes('number').columns)
    imp = IterativeImputer(random_state=423)
    imp.fit(df[num_vars])
    df[num_vars] = imp.transform(df[num_vars])
    return df

In [25]:
# Test function 

zillow = zillow_imputer(zillow)
zillow.isna().sum().sort_values(ascending = False)

census_tractandblock          0
garagecarcnt                  0
lotsizesquarefeet             0
longitude                     0
latitude                      0
heatingorsystemtypeid         0
hashottuborspa                0
garagetotalsqft               0
fireplacecnt                  0
regionid_zip                  0
finishedsquarefeet12          0
decktypeid                    0
buildingqualitytypeid         0
bedroomcnt                    0
bathroomcnt                   0
basementsqft                  0
poolcnt                       0
poolsizesum                   0
propertycountylandusecode     0
roomcnt                       0
unitcnt                       0
yearbuilt                     0
numberofstories               0
structuretaxvaluedollarcnt    0
taxvaluedollarcnt             0
landtaxvaluedollarcnt         0
taxamount                     0
taxdelinquencyflag            0
taxdelinquencyyear            0
logerror                      0
transactiondate               0
county  

In [26]:
# Imputed values are floats - be sure to reset integers to int

# transform float cols to int
int_cols = ['basementsqft', 'bathroomcnt', 'bedroomcnt', 
            'finishedsquarefeet12', 'fireplacecnt',
            'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
            'lotsizesquarefeet', 'poolcnt', 'poolsizesum',
            'roomcnt', 'unitcnt', 'yearbuilt', 'numberofstories',
            'structuretaxvaluedollarcnt', 'taxvaluedollarcnt',
            'landtaxvaluedollarcnt', 'taxamount',
            'taxdelinquencyflag', 'taxdelinquencyyear']

zillow = numeric_to_int(zillow, int_cols)

# transform float cols to object
obj_cols = ['airconditioningtypeid', 'buildingqualitytypeid',
            'decktypeid', 'heatingorsystemtypeid']

zillow = numeric_to_object(zillow, obj_cols)

In [27]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73752 entries, 0 to 73751
Data columns (total 34 columns):
airconditioningtypeid         73752 non-null object
basementsqft                  73752 non-null int64
bathroomcnt                   73752 non-null int64
bedroomcnt                    73752 non-null int64
buildingqualitytypeid         73752 non-null object
decktypeid                    73752 non-null object
finishedsquarefeet12          73752 non-null int64
fireplacecnt                  73752 non-null int64
garagecarcnt                  73752 non-null int64
garagetotalsqft               73752 non-null int64
hashottuborspa                73752 non-null int64
heatingorsystemtypeid         73752 non-null object
latitude                      73752 non-null float64
longitude                     73752 non-null float64
lotsizesquarefeet             73752 non-null int64
poolcnt                       73752 non-null int64
poolsizesum                   73752 non-null int64
propertycountyla

Decide whether to remove the rows or columns of any that cannot be reasonably imputed.

In [28]:
# We handled the null values well enough with imputation, no reason to drop anything

4. Outliers: Original from exercises. Adapt as you see fit.

Write a function that accepts a series (i.e. one column from a data frame) and summarizes how many outliers are in the series. This function should accept a second parameter that determines how outliers are detected, with the ability to detect outliers in 3 ways: IQR, standard deviations (z-score), percentiles)

In [29]:
# IQR outlier detection
def iqr_upper_outliers(s, k):
    '''
    Given a series and a cutoff value, k, returns the upper outliers for the series.

    The values returned will be either 0 (if the point is not an outlier), or a
    number that indicates how far away from the upper bound the observation is.
    '''
    q1, q3 = s.quantile([.25, .75])
    iqr = q3 - q1
    upper_bound = q3 + k * iqr
    return s.apply(lambda x: max([x - upper_bound, 0]))

def add_iqr_outliers(df, out_vars, k):
    '''
    Add a column with the suffix _outliers for all the numeric columns
    in the given dataframe.
    '''
    for col in df[out_vars]:
        df[col + '_iqr'] = iqr_upper_outliers(df[col], k)
    return df

In [30]:
# zscore outlier detection
def zscore_outliers(s, k):
    '''
    Given a series and a cutoff value, k, returns the upper outliers for the series.

    The values returned will be either 0 (if the point is not an outlier), or a
    number that indicates how far away from the upper bound the observation is.
    '''
    outliers = []
    threshold = k
    mean = s.mean()
    std = (s.std() + 0.0001) 
    # just in case the std is 0, add a small amount to avoid the divide by zero error
    
    for x in s:
        z_score= (x - mean)/std 
        if np.abs(z_score) > threshold:
            outliers.append(z_score)
        else:
            outliers.append(0)
    return outliers

def add_zscore_outliers(df, out_vars, k):
    '''
    Add a column with the suffix _outliers for all the numeric columns
    in the given dataframe.
    '''
    for col in df[out_vars]:
        df[col + '_zscore'] = zscore_outliers(df[col], k)
    return df

In [31]:
# percentile outliers
def percentile_outliers(s, k):
    '''
    Given a series and a percent value, k, returns the upper outliers for the series.

    The values returned will be either 0 (if the point is not an outlier), or a
    number that indicates how far away from the upper bound the observation is.
    '''
    upper = 100 - k
    lower_bound, upper_bound = np.percentile(s, [k, upper])
    return s.apply(lambda x: max([x - upper_bound, 0]))

def add_pct_outliers(df, out_vars, k):
    '''
    Add a column with the suffix _outliers for all the numeric columns
    in the given dataframe.
    '''
    for col in df[out_vars]:
        df[col + '_pct'] = percentile_outliers(df[col], k)
    return df

In [32]:
# this function is too intense, can't actually run it
def outliers_df(df, out_vars, style):
    if style == 'iqr':
        add_iqr_outliers(df, out_vars, 1.5)
    elif style == 'zscore':
        add_zscore_outliers(df, out_vars, 3)
    elif style == 'percentile':
        add_pct_outliers(df, out_vars, 10)
    else:
        return print ('valid style options are: iqr, zscore, or percentile')
    return df

Use your function defined above to identify columns where you should handle the outliers.

In [33]:
# too intense
# outliers_df(zillow, outliers, style='iqr')

Write a function that accepts the zillow data frame and removes the outliers. You should make a decision and document how you will remove outliers.

In [34]:
# Columns to look for outliers
outliers = ['basementsqft', 'bathroomcnt', 'bedroomcnt',
            'finishedsquarefeet12', 'garagetotalsqft',
            'lotsizesquarefeet', 'poolsizesum',
            'roomcnt', 'structuretaxvaluedollarcnt',
            'taxvaluedollarcnt', 'landtaxvaluedollarcnt',
            'taxamount']

In [35]:
def outlier_function(df, cols):
    for col in df[cols]:
        mean = df[col].mean()
        std = (df[col].std() + 0.0001) 
        # just in case the std is 0, add a small amount to avoid the divide by zero error
        df = df[df[col] < (mean + (3*std))]
    return df

In [37]:
# test function
outliers_df = outlier_function(zillow, outliers)

In [40]:
outliers_df.head()

,airconditioningtypeid,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,decktypeid,finishedsquarefeet12,fireplacecnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,propertycountylandusecode,roomcnt,unitcnt,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,logerror,transactiondate,county,haversine_distance,regionid_zip,census_tractandblock
0,0,0,2,3,8,0,2107,0,0,0,0,2,"34,222,559.00","-118,617,387.00",9158,1,0,0101,0,1,1972,1,249655,624139,374484,7659,0,0,-0.01,2017-07-07,Los_Angeles,87.51,"96,339.00","60,371,132,321,007.00"
1,1,0,2,4,8,0,1882,0,0,0,0,2,"34,220,261.00","-118,616,409.00",9035,1,0,0101,0,1,1972,1,253000,660000,407000,8123,0,0,0.02,2017-08-29,Los_Angeles,87.60,"96,339.00","60,371,132,321,007.00"
2,1,0,2,4,8,0,1882,0,0,0,0,2,"34,222,491.00","-118,616,854.00",9800,0,0,0100,0,1,1972,1,257591,542923,285332,6673,0,0,0.08,2017-04-04,Los_Angeles,87.47,"96,339.00","60,371,132,321,007.00"
3,0,0,2,3,8,0,1477,0,0,0,0,2,"34,221,864.00","-118,615,739.00",11285,1,0,0101,0,1,1960,1,57968,78031,20063,1116,0,0,-0.04,2017-03-17,Los_Angeles,87.44,"96,339.00","60,371,132,321,008.00"
4,0,0,2,4,8,0,1918,0,0,0,0,2,"34,220,619.00","-118,615,253.00",11239,1,0,0101,0,1,1960,1,167869,415459,247590,5239,0,0,-0.01,2017-03-24,Los_Angeles,87.49,"96,339.00","60,371,132,321,008.00"


In [41]:
outliers_df.shape, zillow.shape

((64841, 34), (73752, 34))

Is there erroneous data you have found that you need to remove or repair? If so, take action.

Are there outliers you want to "squeeze in" to a max value? (e.g. all bathrooms > 6 => bathrooms = 6). If so, make those changes.

## Exploration with Clustering

### Cluster the Target Variable

Perform clustering with logerror as the only feature used in the clustering algorithm. Decide on a number of clusters to use, and store the cluster predictions back onto your data frame as cluster_target. Look at the centroids that were produced in this process. What do they tell you?

Use the produced clusters to help you explore through visualization how logerror relates to other variables. (A common way to do this is to use color to indicate the cluster id, and the other variables can be your x-axis and y-axis. (hint: look at your swarmplot function)).

### Cluster Independent Variables

You should also perform some clustering based on a number of independent variables. Create and evaluate several clustering models based on subsets of the independent variables. Here are some ideas:

- Location, that is, latitude and longitude
- Size (finished square feet)
- Location and size

Be sure to use these new clusters in exploring your data, and interpret what these clusters tell you.

### Test the Significance of Clusters
Use statistical testing methods to determine whether the clusters you have created are significant in terms of their relationship to logerror.

## Modeling

### Feature Engineering

Remove variables that are not needed, wanted, useful, or are redundant.

Add any features you think may be useful.

In [ ]:
# Age = yearbuilt - 2017
# Tax Rate = 
# Avg tax rate by censustractandblockid
# $/sqft
# $/sqft structure only
# $/sqft lot only
# Avg $/sqft by censustractandblockid


Split your data into training and test sets.

In [ ]:
#train, test = wrangle_zillow.split_my_data(df, train_pct)

# what about scaling?

Create subsets of data if you would like to create multiple models and then merge (such as, a different model for each cluster or for each county).

## Model Selection

Train at least 3 different models (a model is different if there are changes in one or more of the following: features, hyper-parameters, algorithm). Create object, fit, predict & evaluate. Use mean absolute error or mean squared error to evaluate. Also, try regression algorithms you have not used before.

Evaluate your best model on your test data set to get an idea of your model's out of sample error.